In [1]:
# Load dataset

from sklearn.model_selection import PredefinedSplit
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import PredefinedSplit
from sklearn.base import TransformerMixin
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn import pipeline
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from keras import backend as K

# Libraries import and data preparation

print(" Always keep your Neurons Happy")
print("             ")

import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
#from dnn_app_utils_v2 import *
import pandas as pd
%matplotlib inline
from pandas import ExcelWriter
from pandas import ExcelFile
%load_ext autoreload
%autoreload 2
from sklearn.utils import resample
import tensorflow as tf
from tensorflow.python.framework import ops
import openpyxl
import keras
import xlsxwriter





from keras.layers import Dense, Dropout
from keras import optimizers





import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
#from dnn_app_utils_v2 import *
import pandas as pd
%matplotlib inline
from pandas import ExcelWriter
from pandas import ExcelFile
%load_ext autoreload
%autoreload 2
from sklearn.utils import resample
import tensorflow as tf
from tensorflow.python.framework import ops
import openpyxl
import keras
import xlsxwriter
from sklearn.preprocessing import Imputer



from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

print(" All the necessary Libraries have been loaded")
print("             ")
print(" The code after this is for loading your data into train, test and CV. Please make sure the path of correct")


train_orig = pd.read_excel(r'are-train.xlsx')
test = pd.read_excel(r'are-test.xlsx')
cv_orig= pd.read_excel(r'are-cv.xlsx')







print(" The data is loaded into train, test and cv")
print("             ")



#Printing sizes before combining all the three

print("size of train : "  + str(np.shape(train_orig)))
print("             ")
print("size of test : "  + str(np.shape(test)))
print("             ")
print("size of cv : "  + str(np.shape(cv_orig)))


print(" Now all the test, train and CV will be combined to make it ready for feature scaling ")
print("             ")



#Combining all the train/test/cv

combined = [train_orig , test, cv_orig]
ar_fulldata = pd.concat(combined)
print("size of train_test_cv: "  + str(np.shape(ar_fulldata)))

Using TensorFlow backend.


 Always keep your Neurons Happy
             
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
 All the necessary Libraries have been loaded
             
 The code after this is for loading your data into train, test and CV. Please make sure the path of correct
 The data is loaded into train, test and cv
             
size of train : (7167, 1446)
             
size of test : (555, 1446)
             
size of cv : (234, 1446)
 Now all the test, train and CV will be combined to make it ready for feature scaling 
             
size of train_test_cv: (7956, 1446)


In [2]:
#######################################################################################################



print(" Now dropping all the rows with NAN values ")
print("             ")

# Dropping all the rows with "NAN" values
ar_fulldata=ar_fulldata.dropna()


print(" Now dropping those columns with zero values entirely or which sums to zero ")
print("             ")

# Dropping all the columns with all zero values
ar_fulldata = ar_fulldata.loc[:, (ar_fulldata != 0).any(axis=0)]



#Separate out the Activity, names and other data
print(" Separating out Activity and Names column ")
print("             ")

ar_fulldata_labels = ar_fulldata.ACTIVITY
ar_fulldata_names = ar_fulldata.NAME
ar_fulldata_input=ar_fulldata.drop(['ACTIVITY', 'NAME'], axis=1)


print(" Calculatig the mean and STD of the input data test and train both together ")
print("             ")
# Describing Statistics of full data, range, mean and STD 
ar_fulldata_input_mean=ar_fulldata_input.mean()
ar_fulldata_input_std=ar_fulldata_input.std()



print(" Dividing the whole train and test on its STD anb sabstracting from the mean ")
print("             ")


#z-score normalizing the whole input data:
ar_fulldata_input_norm = (ar_fulldata_input - ar_fulldata_input_mean)/ar_fulldata_input_std



print(" Checking iif any columns contain NAN values ")
print("             ")

#Checking for the columns with with NAN value.
ar_fulldata_input_norm.loc[:, ar_fulldata_input_norm.isnull().any()]





print(" Adding names and labels to the full data again ")
print("             ")
# Adding names and labels
frames = [ar_fulldata_names,ar_fulldata_labels, ar_fulldata_input_norm]
ar_fulldata_norm = pd.concat(frames,axis=1)

 Now dropping all the rows with NAN values 
             
 Now dropping those columns with zero values entirely or which sums to zero 
             
 Separating out Activity and Names column 
             
 Calculatig the mean and STD of the input data test and train both together 
             
 Dividing the whole train and test on its STD anb sabstracting from the mean 
             
 Checking iif any columns contain NAN values 
             
 Adding names and labels to the full data again 
             


In [3]:
start_train=0
end_train=(np.amax(train_orig.index.values)+1)




train_norm = ar_fulldata_norm.iloc[0:end_train, :]

In [4]:
print(" Diving into test train and cv again after normalization/scaleerization equally all the three ")
print("             ")


start_test=(np.amax(train_orig.index.values)+1)
end_test=(np.amax(train_orig.index.values)+1)+(np.amax(test.index.values)+1)



test=ar_fulldata_norm.iloc[start_test:end_test, :]


 Diving into test train and cv again after normalization/scaleerization equally all the three 
             


In [5]:


start_cv=(np.amax(train_orig.index.values)+1)+(np.amax(test.index.values)+1)
end_cv=(np.amax(train_orig.index.values)+1)+(np.amax(test.index.values)+1)+np.amax(cv_orig.index.values)+1






cv_norm=ar_fulldata_norm.iloc[start_cv:end_cv, :]

In [6]:
print(" adding cv with train ")
print("             ")
#addind cv with train
frames = [train_norm, cv_norm]
train_cv_norm = pd.concat(frames)








print(" Now upsampling the train minority and also shuffeling it ")
print("             ")


# Splitting cv and train before upsampling

from sklearn.model_selection import train_test_split

train, cross_val = train_test_split(train_cv_norm, test_size=0.20, shuffle=False, random_state=41)


train_ones= train['ACTIVITY'].sum()
train_zeros=train.shape[0]-train_ones

cross_val_ones= cross_val['ACTIVITY'].sum()
cross_val_zeros=cross_val.shape[0]-cross_val_ones


# Upsampling the training data to balance it and then shuffeling it again.
from sklearn.utils import shuffle
train_majority = train[train.ACTIVITY==0]
train_minority = train[train.ACTIVITY==1]

train_minority_upsampled = resample(train_minority, 
                                 replace=True,     
                                 n_samples=train_zeros)    
                             
train_upsampled = pd.concat([train_majority, train_minority_upsampled])

print("Trainig set size after upsampling: "  + str(np.shape(train_upsampled )))

train_upsampled=shuffle(train_upsampled)



cross_val_majority = cross_val[cross_val.ACTIVITY==0]
cross_val_minority = cross_val[cross_val.ACTIVITY==1]

cross_val_minority_upsampled = resample(cross_val_minority, 
                                 replace=True,     
                                 n_samples=cross_val_zeros)    
                             
cross_val_upsampled = pd.concat([cross_val_majority, cross_val_minority_upsampled])

print("cross_val set size after upsampling: "  + str(np.shape(cross_val_upsampled )))

cross_val_upsampled=shuffle(cross_val_upsampled)




print(" Dividing into train_x , train_y, test_x and test_y and also dropping the Activity and NAME ")
print("             ")












train_x_upsampled = train_upsampled.drop(['ACTIVITY','NAME'], axis=1)     
train_y_upsampled = train_upsampled.ACTIVITY
train_y_upsampled =train_y_upsampled .reshape(train_y_upsampled.shape[0],1)
train_y_upsampled=pd.DataFrame(train_y_upsampled)



cross_val_x_upsampled = cross_val_upsampled.drop(['ACTIVITY','NAME'], axis=1)     
cross_val_y_upsampled = cross_val_upsampled.ACTIVITY
cross_val_y_upsampled=cross_val_y_upsampled .reshape(cross_val_y_upsampled.shape[0],1)
cross_val_y_upsampled=pd.DataFrame(cross_val_y_upsampled)









test_x = test.drop(['ACTIVITY','NAME'], axis=1)     
test_y = test.ACTIVITY
test_y = test_y .reshape(test_y .shape[0],1)
test_y=pd.DataFrame(test_y)








# Writing test set to exel after normalization


writer = pd.ExcelWriter('test.xlsx',engine='xlsxwriter')
pd.DataFrame(test_x).to_excel(writer,sheet_name='test_x')
pd.DataFrame(test_y).to_excel(writer,sheet_name='test_y')

writer.save()

 adding cv with train 
             
 Now upsampling the train minority and also shuffeling it 
             
Trainig set size after upsampling: (9448, 1267)
cross_val set size after upsampling: (2710, 1267)
 Dividing into train_x , train_y, test_x and test_y and also dropping the Activity and NAME 
             


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:101: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [7]:
# Optimizing the tree paramters with 3 cross CV validation

gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

from sklearn.preprocessing import FunctionTransformer
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator
from sklearn.model_selection import RandomizedSearchCV
frames11 = [train_x_upsampled, cross_val_x_upsampled]
train_cv_x = pd.concat(frames11)

frames22 = [train_y_upsampled, cross_val_y_upsampled]
train_cv_y = pd.concat(frames22)
from scipy.stats import randint as sp_randint
xx=0

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

class fs(TransformerMixin, BaseEstimator):

    def __init__(self, n_estimators=1200):
        self.ss=None
        self.n_estimators = n_estimators
        self.x_new = None
        

        

    def fit(self, X, y):
        m = ExtraTreesClassifier(self.n_estimators)
        m.fit(X,y)
        self.ss = SelectFromModel(m, prefit=True)
        return self

    def transform(self, X):
        self.x_new=self.ss.transform(X)
        print(np.shape(self.x_new))
        global xx
        xx=self.x_new.shape[1]
        
        
        
        return self.x_new
    

    


def create_model(dropout_rate=0.7, activation='sigmoid'):
    print(xx)


    n_x_new=xx
    np.random.seed(6000)
    model_new = Sequential()
    model_new.add(Dense(xx,input_dim=xx ,kernel_initializer='he_uniform', activation='relu'))
    model_new.add(Dense(10, kernel_initializer='he_uniform', activation='relu'))
    model_new.add(Dropout(dropout_rate))
    model_new.add(Dense(1,kernel_initializer='he_uniform', activation='sigmoid'))
    #optimizer = keras.optimizers.Adam(lr=learn_rate)
    model_new.compile(loss='binary_crossentropy',optimizer='adam', metrics=['binary_crossentropy'])

    return model_new



clf=KerasClassifier(build_fn=create_model, epochs=20, batch_size=1000, verbose=1)

model = Pipeline([('fs', fs()),('clf', clf)])

         
param_dist = {'fs__n_estimators': sp_randint (1000, 2000)
              
              
}
        
n_iter_search = 6

grid = RandomizedSearchCV(estimator=model,param_distributions=param_dist,scoring='roc_auc',error_score=0,  n_iter=n_iter_search, n_jobs=1)
grid_result = grid.fit(train_cv_x, train_cv_y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    


c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:37: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


(8105, 597)
597
Epoch 1/20
8105/8105 [==============================] - 1s - loss: 0.7075 - binary_crossentropy: 0.7075     
Epoch 2/20
8105/8105 [==============================] - 0s - loss: 0.6075 - binary_crossentropy: 0.6075     
Epoch 3/20
8105/8105 [==============================] - 0s - loss: 0.5790 - binary_crossentropy: 0.5790     
Epoch 4/20
8105/8105 [==============================] - 0s - loss: 0.5657 - binary_crossentropy: 0.5657     
Epoch 5/20
8105/8105 [==============================] - 0s - loss: 0.5491 - binary_crossentropy: 0.5491     
Epoch 6/20
8105/8105 [==============================] - 0s - loss: 0.5417 - binary_crossentropy: 0.5417     
Epoch 7/20
8105/8105 [==============================] - 0s - loss: 0.5235 - binary_crossentropy: 0.5235     
Epoch 8/20
8105/8105 [==============================] - 0s - loss: 0.5171 - binary_crossentropy: 0.5171     
Epoch 9/20
8105/8105 [==============================] - 0s - loss: 0.5015 - binary_crossentropy: 0.5015     
Epo

c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:37: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


(8105, 606)
606
Epoch 1/20
8105/8105 [==============================] - 0s - loss: 0.6957 - binary_crossentropy: 0.6957     
Epoch 2/20
8105/8105 [==============================] - 0s - loss: 0.6058 - binary_crossentropy: 0.6058     
Epoch 3/20
8105/8105 [==============================] - 0s - loss: 0.5888 - binary_crossentropy: 0.5888     
Epoch 4/20
8105/8105 [==============================] - 0s - loss: 0.5649 - binary_crossentropy: 0.5649     
Epoch 5/20
8105/8105 [==============================] - 0s - loss: 0.5606 - binary_crossentropy: 0.5606     
Epoch 6/20
8105/8105 [==============================] - 0s - loss: 0.5403 - binary_crossentropy: 0.5403     
Epoch 7/20
8105/8105 [==============================] - 0s - loss: 0.5202 - binary_crossentropy: 0.5202     
Epoch 8/20
8105/8105 [==============================] - 0s - loss: 0.5070 - binary_crossentropy: 0.5070     
Epoch 9/20
8105/8105 [==============================] - 0s - loss: 0.5037 - binary_crossentropy: 0.5037     
Epo

c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:37: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


...

(8105, 610)
610
Epoch 1/20
8105/8105 [==============================] - 1s - loss: 0.6832 - binary_crossentropy: 0.6832     
Epoch 2/20
8105/8105 [==============================] - 0s - loss: 0.6136 - binary_crossentropy: 0.6136     
Epoch 3/20
8105/8105 [==============================] - 0s - loss: 0.5810 - binary_crossentropy: 0.5810     
Epoch 4/20
8105/8105 [==============================] - 0s - loss: 0.5638 - binary_crossentropy: 0.5638     
Epoch 5/20
8105/8105 [==============================] - 0s - loss: 0.5536 - binary_crossentropy: 0.5536     
Epoch 6/20
8105/8105 [==============================] - 0s - loss: 0.5355 - binary_crossentropy: 0.5355     
Epoch 7/20
8105/8105 [==============================] - 0s - loss: 0.5194 - binary_crossentropy: 0.5194     
Epoch 8/20
8105/8105 [==============================] - 0s - loss: 0.5082 - binary_crossentropy: 0.5082     
Epoch 9/20
8105/8105 [==============================] - 0s - loss: 0.5001 - binary_crossentropy: 0.5001     
Epo

c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:37: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


(8106, 624)
624
Epoch 1/20
8106/8106 [==============================] - 1s - loss: 0.7213 - binary_crossentropy: 0.7213     
Epoch 2/20
8106/8106 [==============================] - 0s - loss: 0.6183 - binary_crossentropy: 0.6183     
Epoch 3/20
8106/8106 [==============================] - 0s - loss: 0.5938 - binary_crossentropy: 0.5938     
Epoch 4/20
8106/8106 [==============================] - 0s - loss: 0.5806 - binary_crossentropy: 0.5806     
Epoch 5/20
8106/8106 [==============================] - 0s - loss: 0.5594 - binary_crossentropy: 0.5594     
Epoch 6/20
8106/8106 [==============================] - 0s - loss: 0.5400 - binary_crossentropy: 0.5400     
Epoch 7/20
8106/8106 [==============================] - 0s - loss: 0.5318 - binary_crossentropy: 0.5318     
Epoch 8/20
8106/8106 [==============================] - 0s - loss: 0.5238 - binary_crossentropy: 0.5238     
Epoch 9/20
8106/8106 [==============================] - 0s - loss: 0.5063 - binary_crossentropy: 0.5063     
Epo

c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:37: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


(12158, 615)
615
Epoch 1/20
12158/12158 [==============================] - 1s - loss: 0.6618 - binary_crossentropy: 0.6618     
Epoch 2/20
12158/12158 [==============================] - 0s - loss: 0.6002 - binary_crossentropy: 0.6002     
Epoch 3/20
12158/12158 [==============================] - 0s - loss: 0.5713 - binary_crossentropy: 0.5713     
Epoch 4/20
12158/12158 [==============================] - 0s - loss: 0.5544 - binary_crossentropy: 0.5544     
Epoch 5/20
12158/12158 [==============================] - 0s - loss: 0.5325 - binary_crossentropy: 0.5325     
Epoch 6/20
12158/12158 [==============================] - 0s - loss: 0.5163 - binary_crossentropy: 0.5163     
Epoch 7/20
12158/12158 [==============================] - 0s - loss: 0.5017 - binary_crossentropy: 0.5017     
Epoch 8/20
12158/12158 [==============================] - 0s - loss: 0.4914 - binary_crossentropy: 0.4914     
Epoch 9/20
12158/12158 [==============================] - 0s - loss: 0.4808 - binary_crossentro

In [67]:
# Selecting the new features based on the tree paramters optimization



print(" Now is the turn to select important features basesd on tree search")
print("             ")


print(" Combining train_upsampled and test for prepairing it for important feature selection")
print("             ")
frames11 = [train_x_upsampled, cross_val_x_upsampled]
train_cv_x = pd.concat(frames11)

frames22 = [train_y_upsampled, cross_val_y_upsampled]
train_cv_y = pd.concat(frames22)





### Code for important feature selection




from sklearn.metrics import roc_auc_score, roc_curve
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel


#grid_result.best_params_["fs__n_estimators"]

    
m = ExtraTreesClassifier(n_estimators=1020)
#m = ExtraTreesClassifier(150)


m.fit(train_cv_x,train_cv_y)


sel = SelectFromModel(m,threshold='1*mean', prefit=True)

print(" Getting features names ")
print("  ")
feature_idx = sel.get_support()
feature_name = train_cv_x.columns[feature_idx]
feature_name =pd.DataFrame(feature_name )

X_new = sel.transform(train_cv_x)
X_new =pd.DataFrame(X_new)


print(" adding names and important feature values ")
print("  ")
X_new.columns = feature_name



print(" dividing the imporrtant features into train and test ")
print("  ")




writer = pd.ExcelWriter('feature_name.xlsx',engine='xlsxwriter')
pd.DataFrame(feature_name).to_excel(writer,sheet_name='feature_name')


writer.save()








xls = pd.ExcelFile("test.xlsx")
test_x = pd.read_excel(xls, 'test_x')
test_y = pd.read_excel(xls, 'test_y') 

 
feature_name=feature_name.T
feature_name.columns = feature_name.iloc[0]
feature_name.reindex(feature_name.index.drop(0))
train_selected_x=train_cv_x[train_cv_x.columns.intersection(feature_name.columns)]
test_selected_x=test_x[test_x.columns.intersection(feature_name.columns)]


train_selected_x=train_selected_x.as_matrix()
test_selected_x=test_selected_x.as_matrix()

train_selected_y=train_cv_y.as_matrix()

test_selected_y=test_y.as_matrix()


writer = pd.ExcelWriter('test_selected.xlsx',engine='xlsxwriter')
pd.DataFrame(test_selected_x).to_excel(writer,sheet_name='test_selected_x')
pd.DataFrame(test_selected_y).to_excel(writer,sheet_name='test_selected_y')

writer.save()




########################################################################################################################

 Now is the turn to select important features basesd on tree search
             
 Combining train_upsampled and test for prepairing it for important feature selection
             


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


 Getting features names 
  
 adding names and important feature values 
  
 dividing the imporrtant features into train and test 
  


In [69]:
#drop_best

print(train_selected_x.shape[1])

model_new_1 = Sequential()
#n_x_new=train_selected_x.shape[1]
n_x_new=train_selected_x.shape[1]
model_new_1.add(Dense(n_x_new, input_dim=n_x_new, kernel_initializer ='he_normal', activation='sigmoid'))


model_new_1.add(Dense(10, kernel_initializer='he_normal', activation='sigmoid'))
model_new_1.add(Dropout(0.8))

#dp=0.4
#epochs=10
#batch_size=3000
#input_neueon=1000
#ini=he_normal
model_new_1.add(Dense(1, kernel_initializer='he_normal', activation='sigmoid'))




model_new_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_crossentropy'])

seed = 7000
np.random.seed(seed)


model_new_1.fit(train_selected_x, train_selected_y, epochs=20, batch_size=1024)

pred_test_1 = model_new_1.predict(test_selected_x)
auc_test_1 = roc_auc_score(test_selected_y, pred_test_1)
print ("AUROC_test: " + str(auc_test_1))

print("  ") 
model_json = model_new_1.to_json()
with open("1_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_new_1.save_weights("1_model.h5")
print("Saved model to disk")
print("  ")

615
Epoch 1/20
12158/12158 [==============================] - 4s 348us/step - loss: 0.6864 - binary_crossentropy: 0.6864
Epoch 2/20
12158/12158 [==============================] - 3s 237us/step - loss: 0.6481 - binary_crossentropy: 0.6481
Epoch 3/20
12158/12158 [==============================] - 3s 229us/step - loss: 0.6328 - binary_crossentropy: 0.6328
Epoch 4/20
12158/12158 [==============================] - 3s 237us/step - loss: 0.6213 - binary_crossentropy: 0.6213
Epoch 5/20
12158/12158 [==============================] - 3s 208us/step - loss: 0.6128 - binary_crossentropy: 0.6128
Epoch 6/20
12158/12158 [==============================] - 3s 208us/step - loss: 0.6072 - binary_crossentropy: 0.6072
Epoch 7/20
12158/12158 [==============================] - 3s 206us/step - loss: 0.6025 - binary_crossentropy: 0.6025
Epoch 8/20
12158/12158 [==============================] - 3s 206us/step - loss: 0.5937 - binary_crossentropy: 0.5937
Epoch 9/20
12158/12158 [==============================] - 3s

In [77]:
#drop_best

print(train_selected_x.shape[1])

model_new_2 = Sequential()
#n_x_new=train_selected_x.shape[1]
n_x_new=train_selected_x.shape[1]
model_new_2.add(Dense(n_x_new, input_dim=n_x_new, kernel_initializer ='he_normal', activation='sigmoid'))


model_new_2.add(Dense(10, kernel_initializer='he_normal', activation='sigmoid'))
model_new_2.add(Dropout(0.5))


model_new_2.add(Dense(1, kernel_initializer='he_normal', activation='sigmoid'))




model_new_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_crossentropy'])

seed = 7000
np.random.seed(seed)


model_new_2.fit(train_selected_x, train_selected_y, epochs=20, batch_size=1024
               )


pred_test_2 = model_new_2.predict(test_selected_x)
auc_test_2 = roc_auc_score(test_selected_y, pred_test_2)
print ("AUROC_test: " + str(auc_test_2))





print("  ") 
model_json = model_new_2.to_json()
with open("2_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_new_2.save_weights("2_model.h5")
print("Saved model to disk")
print("  ")

615
Epoch 1/20
12158/12158 [==============================] - 4s 339us/step - loss: 0.6493 - binary_crossentropy: 0.6493
Epoch 2/20
12158/12158 [==============================] - 3s 208us/step - loss: 0.6075 - binary_crossentropy: 0.6075
Epoch 3/20
12158/12158 [==============================] - 3s 232us/step - loss: 0.5840 - binary_crossentropy: 0.5840
Epoch 4/20
12158/12158 [==============================] - 3s 218us/step - loss: 0.5684 - binary_crossentropy: 0.5684
Epoch 5/20
12158/12158 [==============================] - 3s 208us/step - loss: 0.5563 - binary_crossentropy: 0.5563
Epoch 6/20
12158/12158 [==============================] - 2s 204us/step - loss: 0.5454 - binary_crossentropy: 0.5454
Epoch 7/20
12158/12158 [==============================] - 2s 204us/step - loss: 0.5344 - binary_crossentropy: 0.5344
Epoch 8/20
12158/12158 [==============================] - 2s 203us/step - loss: 0.5218 - binary_crossentropy: 0.5218
Epoch 9/20
12158/12158 [==============================] - 2s

In [78]:
#pred_train = model_new_1.predict(train_selected_x)
pred_test_1 = model_new_1.predict(test_selected_x)
pred_test_2 = model_new_2.predict(test_selected_x)

#auc_train = roc_auc_score(train_selected_y, pred_train)

#### Averaging the ouput 
pred_test=(pred_test_1+pred_test_2)/2


auc_test = roc_auc_score(test_selected_y, pred_test)


print ("AUROC_test: " + str(auc_test))

AUROC_test: 0.82706057987


In [84]:
#drop_best

print(train_selected_x.shape[1])

model_new_3 = Sequential()
#n_x_new=train_selected_x.shape[1]
n_x_new=train_selected_x.shape[1]
model_new_3.add(Dense(n_x_new, input_dim=n_x_new, kernel_initializer ='he_normal', activation='sigmoid'))


model_new_3.add(Dense(10, kernel_initializer='he_normal', activation='sigmoid'))
model_new_3.add(Dropout(0.8))

#dp=0.4
#epochs=10
#batch_size=3000
#input_neueon=1000
#ini=he_normal
model_new_3.add(Dense(1, kernel_initializer='he_normal', activation='sigmoid'))




model_new_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_crossentropy'])

seed = 7000
np.random.seed(seed)


model_new_3.fit(train_selected_x, train_selected_y, epochs=20, batch_size=1024)

pred_test_3 = model_new_3.predict(test_selected_x)
auc_test_3 = roc_auc_score(test_selected_y, pred_test_3)
print ("AUROC_test: " + str(auc_test_3))




print("  ") 
model_json = model_new_3.to_json()
with open("3_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_new_3.save_weights("3_model.h5")
print("Saved model to disk")
print("  ")

615
Epoch 1/20
12158/12158 [==============================] - 4s 348us/step - loss: 0.6864 - binary_crossentropy: 0.6864
Epoch 2/20
12158/12158 [==============================] - 3s 209us/step - loss: 0.6481 - binary_crossentropy: 0.6481
Epoch 3/20
12158/12158 [==============================] - 3s 210us/step - loss: 0.6328 - binary_crossentropy: 0.6328
Epoch 4/20
12158/12158 [==============================] - 3s 210us/step - loss: 0.6213 - binary_crossentropy: 0.6213
Epoch 5/20
12158/12158 [==============================] - 3s 210us/step - loss: 0.6128 - binary_crossentropy: 0.6128
Epoch 6/20
12158/12158 [==============================] - 3s 209us/step - loss: 0.6071 - binary_crossentropy: 0.6071
Epoch 7/20
12158/12158 [==============================] - 3s 210us/step - loss: 0.6025 - binary_crossentropy: 0.6025
Epoch 8/20
12158/12158 [==============================] - 3s 220us/step - loss: 0.5937 - binary_crossentropy: 0.5937
Epoch 9/20
12158/12158 [==============================] - 3s

In [85]:
#pred_train = model_new_1.predict(train_selected_x)
pred_test_1 = model_new_1.predict(test_selected_x)
pred_test_2 = model_new_2.predict(test_selected_x)
pred_test_3 = model_new_3.predict(test_selected_x)

#auc_train = roc_auc_score(train_selected_y, pred_train)

#### Averaging the ouput 
pred_test=(pred_test_1+pred_test_2+pred_test_3)/3


auc_test = roc_auc_score(test_selected_y, pred_test)


print ("AUROC_test: " + str(auc_test))

AUROC_test: 0.827479636468


In [86]:
#drop_best

print(train_selected_x.shape[1])

model_new_4 = Sequential()
#n_x_new=train_selected_x.shape[1]
n_x_new=train_selected_x.shape[1]
model_new_4.add(Dense(n_x_new, input_dim=n_x_new, kernel_initializer ='he_normal', activation='sigmoid'))


model_new_4.add(Dense(10, kernel_initializer='he_normal', activation='sigmoid'))
model_new_4.add(Dropout(0.8))

#dp=0.4
#epochs=10
#batch_size=3000
#input_neueon=1000
#ini=he_normal
model_new_4.add(Dense(1, kernel_initializer='he_normal', activation='sigmoid'))




model_new_4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_crossentropy'])

seed = 7000
np.random.seed(seed)


model_new_4.fit(train_selected_x, train_selected_y, epochs=20, batch_size=1024)

pred_test_4 = model_new_4.predict(test_selected_x)
auc_test_4 = roc_auc_score(test_selected_y, pred_test_4)
print ("AUROC_test: " + str(auc_test_4))





print("  ") 
model_json = model_new_4.to_json()
with open("4_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_new_4.save_weights("4_model.h5")
print("Saved model to disk")
print("  ")

615
Epoch 1/20
12158/12158 [==============================] - 4s 342us/step - loss: 0.6864 - binary_crossentropy: 0.6864
Epoch 2/20
12158/12158 [==============================] - 3s 209us/step - loss: 0.6481 - binary_crossentropy: 0.6481
Epoch 3/20
12158/12158 [==============================] - 2s 204us/step - loss: 0.6328 - binary_crossentropy: 0.6328
Epoch 4/20
12158/12158 [==============================] - 3s 208us/step - loss: 0.6213 - binary_crossentropy: 0.6213
Epoch 5/20
12158/12158 [==============================] - 3s 208us/step - loss: 0.6128 - binary_crossentropy: 0.6128
Epoch 6/20
12158/12158 [==============================] - 3s 207us/step - loss: 0.6071 - binary_crossentropy: 0.6071
Epoch 7/20
12158/12158 [==============================] - 3s 210us/step - loss: 0.6025 - binary_crossentropy: 0.6025
Epoch 8/20
12158/12158 [==============================] - 3s 206us/step - loss: 0.5937 - binary_crossentropy: 0.5937
Epoch 9/20
12158/12158 [==============================] - 3s

In [87]:
#pred_train = model_new_1.predict(train_selected_x)
pred_test_1 = model_new_1.predict(test_selected_x)
pred_test_2 = model_new_2.predict(test_selected_x)
pred_test_3 = model_new_3.predict(test_selected_x)
pred_test_4 = model_new_4.predict(test_selected_x)
#auc_train = roc_auc_score(train_selected_y, pred_train)

#### Averaging the ouput 
pred_test=(pred_test_1+pred_test_2+pred_test_3+pred_test_4)/4


auc_test = roc_auc_score(test_selected_y, pred_test)


print ("AUROC_test: " + str(auc_test))








AUROC_test: 0.827741546843
